In [1]:
from transformers import pipeline

In [2]:
tanslator=pipeline("translation_en_to_zh",model='Helsinki-NLP/opus-mt-en-zh')

In [1]:
import torch
import os
import lasio
from lasio import LASFile
from torch.utils.data import Dataset

import pickle
from copy import deepcopy

# 1.原始数据读取


In [4]:
def getfilenamelist(path: str, **kwargs):
    # func same as 'ls' in linux or 'dir' in windows
    extl = kwargs.get("extl")
    Filelist: list[str] = []
    path = os.path.abspath(path)
    for home, dirs, files in os.walk(path):
        for filename in files:
            if extl:
                _, file_extension = os.path.splitext(filename)
                if file_extension in extl:
                    Filelist.append(os.path.join(home, filename))
            else:
                Filelist.append(os.path.join(home, filename))
    return Filelist

In [5]:
def concatlist(listin: list[list]):
    listout = []
    for i in listin:
        listout.extend(i)
    return listout

In [6]:
def dictinverse(inp: dict[str, list]): # inverse dict key and value
    output: dict[str, str] = {}
    for key in inp.keys():
        for value in inp[key]:
            output[value] = key
    return output

In [7]:
rawlas_dir = "../rawlas"
las_name_list = getfilenamelist(rawlas_dir, extl=[".las"])

In [24]:
class MyLas(lasio.LASFile): 
    '''
    las类，继承自lasio，
    '''
    def __init__(self,file):
        super(MyLas,self).__init__(file)
        self.well.WELL=os.path.splitext(os.path.basename(file))[0]
        
    def restep(self,step):
        new_las=self
        return new_las
    
    def cut_depth(self,depth_start,depth_end):
        df=self.df()
        cut_df = df.loc[(df.index >= depth_start) & (df.index <= depth_end)]
        self.set_data(cut_df)
        return self
    
    def cut_curve(self,curve_name):
        idx=self.df()[curve_name].dropna(axis=0,how='any').index
        return self.cut_depth(idx.min(),idx.max())


In [23]:
def getLasListWithTag(las_name_list: list[str],oil_curve_tag:str, **kwargs):
    '''
    筛选包含tag的曲线
    '''
    rownums = kwargs.get("rownums") or -1
    laslist: list[MyLas] = []
    for fn in las_name_list[0:rownums]:
        las = MyLas(fn)
        if oil_curve_tag in las.keys():
            laslist.append(las)
    return laslist

In [22]:
las_list=getLasListWithTag(las_name_list,'SW')
las

In [197]:
# 保存las_list
with open('./data/las_list.pkl','wb') as f:
    pickle.dump(las_list,f)

In [10]:
# 读取las_list
import pickle
with open('./data/las_list.pkl','rb') as f:
    las_list=pickle.load(f)

In [187]:
las_list[0].well.WELL='test'

In [204]:
las_list[4].well.WELL

'guan69-16H'

# 2.曲线选取

In [19]:
curves_dict = {
    "DEPTH":["DEPTH","DEP","DEPT"],
    "SP": ["SP"],
    "GR": ["GR"],
    "AC": ["AC", "DT", "P_AC"],
    "RT": ["RT", "RD", "RILD", "ILD", "R045", "RA45"],
    "RXO": ["RXO", "RA04", "RA25", "R025", "RIL"],
    "POR": ["POR"],
    "PORF": ["PORF"],
    "PORW": ["PORW"],

}

In [17]:
def washCurvesWithDict(las:MyLas,curves_dict: dict, **kwargs):
    las_copy=deepcopy(las)
    for laskey in las.keys():
        if laskey not in concatlist(curves_dict.values()):
            las_copy.delete_curve(laskey)
    return las_copy

In [89]:
# 保存las_list
with open('./data/las_list.pkl','wb') as f:
    pickle.dump(las_list,f)

In [11]:
# 读取las_list
import pickle
with open('./data/las_list.pkl','rb') as f:
    las_lists=pickle.load(f)

# 2.曲线绘制

In [12]:
import plotly.graph_objs as go
import plotly.offline as po

In [13]:
def curvesplot(df, columns=None,yaxis=None, space=0.05,title="",mode='lines', show=1): # plot curves in df
    if columns == None:
        columns = df.columns.to_list()
    if yaxis==None:
        yaxis=df.index
    else:
        if yaxis in columns:
            columns.remove(yaxis)
        yaxis=df[yaxis]

    leny = len(columns)
    curvelist = dict()
    layout = dict()
    nax = 0

    for i in columns:
        nax = nax+1
        curvelist[i] = go.Scattergl(
            x=df[i],
            y=yaxis,
            xaxis='x' if nax == 1 else 'x'+str(nax),
            name=i,
            mode=mode
        )
        layout['xaxis' if nax == 1 else 'xaxis'+str(nax)] = {
            "domain": [(nax-1)/leny+space/leny, nax/leny-space/leny],
            'title': i
        }
        layout['yaxis' if nax == 1 else 'yaxis'+str(nax)] = {
            "anchor": 'x' if nax == 1 else 'x'+str(nax),
            'autorange': 'reversed',
            'showspikes': True
        }
    layout['title']=title
    fig = go.Figure(
        data=[i for i in curvelist.values()],
        layout=layout
    )
    if show == 1:
        po.plot(fig)
    return fig


In [14]:
def laslisttodict(laslist):
    lasdict=dict()
    for i in laslist:
        lasdict[i.well.WELL]=i
    return lasdict

In [15]:
las_dict=laslisttodict(las_list)

In [20]:
testlas=washCurvesWithDict(las_list[5],curves_dict).cut_curve('POR')
testdf=testlas.df()
testdf['O-A']=testdf['POR']-testdf['PORW']
testdf['O-F']=testdf['PORF']-testdf['PORW']

In [208]:
testlas.well.WELL

'guan69-2-1'

In [21]:
curvesplot(testdf,title=testlas.well.WELL)

In [170]:
testlas.df()['AC'].median()

305.2845

# 2.数据处理 - 曲线切割


In [94]:
testlas=las_lists[2]
curvesplot(testlas.df())

In [73]:
testlas.cut_curve('SO')

KeyError: 'SO'

In [68]:
lasio.read(lasio.read(las_name_list[0]))

AttributeError: 'int' object has no attribute 'read'

In [67]:
a=MyLas(testlas)

AttributeError: 'int' object has no attribute 'read'

In [3]:
class LasDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.image_paths = [
            os.path.join(data_dir, f)
            for f in os.listdir(data_dir)
            if f.endswith(".las")
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        # perform any required preprocessing on the image
        # ...
        # convert the image to a PyTorch tensor
        image = transforms.ToTensor()(image)
        return image

In [15]:
print(os.path.realpath(r"."), os.path.abspath("../rawlas"))

F:\mygitproj\codespaces-jupyter\notebooks f:\mygitproj\codespaces-jupyter\rawlas
